In [20]:
!pip install -U kaleido
!pip install -U tensorflow
!pip install -U vectorbt

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/527.5 kB ? eta -:--:--
   --- ------------------------------------ 41.0/527.5 kB ? eta -:--:--
   -------- ------------------------------- 112.6/527.5 kB 1.3 MB/s eta 0:00:01
   ---------- ----------------------------- 143.4/527.5 kB 1.1 MB/s eta 0:00:01
   ----------------- ---------------------- 225.3/527.5 kB 1.3 MB/s eta 0:00:01
   ------------------------ --------------- 317.4/527.5 kB 1.3 MB/s eta 0:00:01
   ------------------------------- -------- 419.8/527.5 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------  522.2/527.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 527.5/527.5 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.8 MB 2.4 MB/s eta 0:00:02


In [40]:
import pandas as pd
import numpy as np
import talib as ta

import plotly.graph_objects as go
from typing import Optional

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

import vectorbt as vbt
import glob
import os

In [11]:
# Загрузка и подготовка данных
df = prepare_data('data/AAVE_USDT_15m_candles.csv', multiplier=2)
tp_profit = df[df['target'] == 1]['returns'].mean()
sl_loss = df[df['target'] == -1]['returns'].abs().mean()
print(f"Средний TP: {tp_profit:.4%}, Средний SL: {sl_loss:.4%}")
print(df.target.value_counts())
    
# Визуализация
viz_df = df[65:104]
fig = create_candlestick_chart(viz_df)
add_levels(fig, viz_df)
add_target_annotations(fig, viz_df)
add_trade_lines(fig, viz_df)  # Заменяем предыдущие аннотации
    
fig.show()

Средний TP: 0.0078%, Средний SL: 0.3570%
target
 0    26468
-1     4413
 1     4055
Name: count, dtype: int64


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """Динамический график изменения learning rate с "прогревом" (warm-up)."""

    def init(self, initial_lr=1e-4, warmup_steps=1000):
        super().init()
        self.initial_lr = initial_lr
        self.warmup_steps = warmup_steps
    

    def __call__(self, step):
        return tf.minimum(
            self.initial_lr * (step / self.warmup_steps)**0.5,
            self.initial_lr
        )
    
def weighted_asymmetric_loss(y_true, y_pred):
    """
    Асимметричная взвешенная кросс-энтропия для задачи классификации направления.
    Ожидается, что y_true содержит one-hot вектора (shape [batch, 3]), а y_pred — вероятности (softmax).
    """
    # Определяем индексы истинного класса
    true_idx = tf.argmax(y_true, axis=1)
    
    # Присваиваем веса: 1.5 для класса "Падение" (index=2),
    # 0.7 для "Стабильность" (index=1), 1.0 для "Рост" (index=0)
    weights = tf.where(true_idx == 2,           # Класс "Падение" 1.5,
                       tf.where(true_idx == 1, # Класс "Стабильность" 0.7, 1.0 # Класс "Рост" 
                        )
    )
    ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

    return tf.reduce_mean(ce * weights)
    

def mare_loss(y_true, y_pred):
    """
    MARE (Mean Absolute Relative Error). Подходит для процентных изменений.
    """
    return tf.reduce_mean( tf.abs((y_true - y_pred) / (y_true + 1e-8)) )

def temporal_consistency_loss(y_pred):
    """
    Пример регуляризации временной согласованности (демонстрация).
    Для реальных временных рядов чаще обрабатывают последовательные выходы.
    """
    # В данном упрощённом примере возвращаем 0, чтобы не вносить вклад в итоговую функцию потерь.
    return 0.0


class CustomLoss(tf.keras.losses.Loss):
    """
    Комбинированная функция потерь, объединяющая:
    - weighted_asymmetric_loss для направления
    - mare_loss для уровней High/Low
    - опционально temporal_consistency_loss
    """
    
    def init(self, alpha=0.3, beta=0.2, name='custom_loss'):
        super().init(name=name)
        self.alpha = alpha # Вес модельного вклада для прогноза High/Low
        self.beta = beta # Вес временной регуляризации (демонстрация)
    
    def call(self, y_true, y_pred):
        # y_pred — список [direction_pred, high_pred, low_pred]
        direction_pred = y_pred[0]
        high_pred = y_pred[1]
        low_pred = y_pred[2]

        # y_true — список [direction_true, high_true, low_true]
        direction_true = y_true[0]
        high_true = y_true[1]
        low_true = y_true[2]

        # Основные компоненты потерь
        ce_loss = weighted_asymmetric_loss(direction_true, direction_pred)
        hl_loss = 0.5 * (mare_loss(high_true, high_pred) + mare_loss(low_true, low_pred))
        temp_loss = self.beta * temporal_consistency_loss(direction_pred)  # Пример

        total_loss = ce_loss + self.alpha * hl_loss + temp_loss
        return total_loss

,-1,0,1,open,high,low,close
0,False,True,False,51.90,52.00,51.80,51.80
1,False,True,False,51.70,51.80,51.70,51.80
2,False,True,False,51.80,51.80,51.70,51.80
3,False,True,False,51.70,51.80,51.70,51.80
4,False,True,False,51.80,51.80,51.80,51.80
...,...,...,...,...,...,...,...
34931,True,False,False,112.08,112.52,111.85,112.41
34932,False,False,True,112.43,112.48,111.33,112.09
34933,False,True,False,112.08,112.95,112.02,112.45
34934,False,True,False,112.40,112.51,112.03,112.15


In [ ]:
def prepare_data_for_train(data, window_size=30):
    """
    Подготовка данных: Превращаем [samples, features] ->
    [samples, window_size, features] для LSTM. Предполагается, что в data:
    - первые три столбца для one-hot направления,
    - четвертый столбец — High,
    - пятый столбец — Low,
    - остальные — вспомогательные фичи (open, close, atr и т.д.).
    """
    
    X, y_dir, y_high, y_low = [], [], [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size : i]) # direction (one-hot) в колонках 0..2
        y_dir.append(data[i, 0:3]) # High — column 3, Low — column 4
        y_high.append(data[i, 3])
        y_low.append(data[i, 4])
        X = np.array(X)
        y_dir = np.array(y_dir)
        y_high = np.array(y_high).reshape(-1, 1)
        y_low = np.array(y_low).reshape(-1, 1)
        return X, [y_dir, y_high, y_low]
